# 화장품 추천 모델 : CF

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import math

import re

from tqdm import tqdm_notebook
import os
import warnings  
warnings.filterwarnings('ignore')

# 그래픽 관련
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz
# 그래프 문자 깨지는 것 대처
import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline

# 추천시스템
import surprise
from surprise import SVD, evaluate
from surprise import NMF
from surprise import KNNBasic

# 1. 학습시키기 위한 데이터 셋 만들기

## ■data : raw_data(15881 rows, 25 columns)

In [32]:
data = pd.read_excel('data/prepro6.xlsx')

In [33]:
print(data.shape)
data.head(1)

(15879, 26)


,No.,user,user_reviews,review,product,brand,pr_per_vol100,rating_one,rating_aver,date,...,tags,vol_price,vol,price,7gra_ingre,20caution_ingre,allergy_ingre,forji_ingre,forgun_ingre,min_ingre
0,8035,꽃찐이,1,ㅠㅠㅠㅠㅠ 페이스북에서 광고를 너무해서 거르고싶었는데 정수리 탈모가 시작되서 급하게...,폴리젠 샴푸,닥터포헤어,7400,5,3.67,2018.12.9,...,"['두피건강', '두피피지관리', '모근강화', '모발건강', '탈모예방']","500ml / 37,000원",500,37000,1,4,0,2,3,4


In [34]:
del data['No.']

In [35]:
data.columns.unique()

Index(['user', 'user_reviews', 'review', 'product', 'brand', 'pr_per_vol100',
       'rating_one', 'rating_aver', 'date', 'year', 'month', 'gender', 'age',
       'type', 'joiner', 'tags', 'vol_price', 'vol', 'price', '7gra_ingre',
       '20caution_ingre', 'allergy_ingre', 'forji_ingre', 'forgun_ingre',
       'min_ingre'],
      dtype='object')

## 리뷰 2개 이상인 user만 선택하기

In [36]:
data2 = data[data['user_reviews']>=2]

In [37]:
print(data2.shape)
data2.user_reviews.unique()

(8932, 25)


array([ 3,  2,  7,  5,  4,  6,  8, 10, 14, 13, 12,  9, 18, 11, 17, 15],
      dtype=int64)

## ■df : 사용자,제품명,평점 data(8934 rows, 3 columns)

In [38]:
df = data2[['user', 'product', 'rating_one']]

In [39]:
print(df.shape)
df.head(10)

(8932, 3)


,user,product,rating_one
3,내제잘,실크리페어 리커버리 샴푸,2
5,수정222,내추럴 발효초 샴푸,2
6,미묘뮤몌,퍼퓸 퓨어 브리즈 샴푸,1
7,속이알차니,퍼펙트 리페어 샴푸,1
8,볼뤼,그린티민트 프레시 샴푸,5
9,방노방노,퍼펙트 세럼 샴푸 오리지널,2
10,랑꿍,실크리페어 샤이닝 샴푸,3
12,tjdudd,허니 앤 마카다미아 네이처샴푸 베이비파우더,5
13,이브엄마,드라이샴푸 클린&클래식 오리지널,2
15,꽃집여자,녹차실감 샴푸액 지성용,3


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8932 entries, 3 to 15875
Data columns (total 3 columns):
user          8932 non-null object
product       8932 non-null object
rating_one    8932 non-null int64
dtypes: int64(1), object(2)
memory usage: 279.1+ KB


In [41]:
# 판다스를 딕셔너리로 만드는 함수
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.ix[:,1:]) for k,g in grouped}
    return d

## ■df_dict : df의 딕셔너리{유저이름 : {제품명 : 평점}

In [42]:
df_dict = recur_dictify(df)

In [43]:
df_dict

{7: {'PRO-V 극손상 케어 샴푸': 2,
  '[단종] 데미지케어 샴푸NC': 3,
  '[단종] 엘레강스&센슈얼 퍼퓸 샴푸': 2,
  '[단종] 포티샤 스티뮬레이팅 샴푸': 4,
  '데미지 클리닉 샴푸': 4},
 7.7: {'[단종] 아르간 에센셜 딥 케어 샴푸': 5, '녹차실감 샴푸액 중건성용': 5},
 10: {'녹차실감 샴푸액 중건성용': 4, '안티에이징 샴푸': 3, '퍼펙트 세럼 샴푸 오리지널': 2},
 92: {'그린티민트 프레시 샴푸': 4,
  '로즈메리 민트 퓨리파잉 샴푸': 4,
  '벨벳 프로틴 본드 샴푸(뉴트리 인젝션 단백질 본드 샴푸/약산성)': 4,
  '실리콘무첨가 리페어 샴푸': 5,
  '아쿠아 퓨어 샴푸': 4,
  '옐로우 카모마일컬렉션 샴푸': 3,
  '핑크 로즈 샴푸': 3},
 272: {'AHS 모이스처+옥시전 샴푸 (산소샴푸)': 5, '아임 트루 내추럴 샴푸': 3},
 726: {'[단백질샴푸] 알베르 헤어 샴푸': 2, '드라이샴푸 클린&클래식 오리지널': 4},
 829: {'[단종] 민트 프레쉬 샴푸': 4, '쿨멘솔 샴푸': 4, '폴리젠 샴푸': 5},
 1119: {'에델바이스 샴푸': 4, '올뉴TS샴푸': 3},
 1262: {'[단종] 로즈마리 스칼프 스케일링 샴푸': 3, '실크리페어 리커버리 샴푸': 2},
 2221: {'[단종] 민트 프레쉬 샴푸': 4, '핑크 로즈 샴푸': 3},
 2976: {'[단종] 로즈마리 스칼프 스케일링 샴푸': 3,
  '실리콘무첨가 리페어 샴푸': 2,
  '아임 트루 내추럴 샴푸': 2,
  '폴리젠 샴푸': 3,
  '피토테라피 천연 샴푸': 3},
 10000: {'녹차실감 샴푸액 지성용': 4, '퍼퓸 샴푸 포에버 피오니': 2},
 11077: {'드라이샴푸 클린&클래식 오리지널': 3, '앱솔루트 샴푸': 4},
 20202: {'실리콘 프리 안티 댄드러프 샴푸': 4, '퍼퓸 퓨어 브리즈 샴푸': 1},
 28428: {'[

In [44]:
df_dict[7.7]

{'[단종] 아르간 에센셜 딥 케어 샴푸': 5, '녹차실감 샴푸액 중건성용': 5}

In [45]:
name_list = []      # 사용자 목록을 담을 리스트   # 중복 가능
cos_set = set()     # 화장품 목록을 담을 set      # 중복 불가

In [46]:
for user_key in df_dict:
    print(user_key)

7
7.7
10
92
272
726
829
1119
1262
2221
2976
10000
11077
20202
28428
43586
43811
101036
221324
357126
373737
9999999
64946458
#soso
#달덩이
(박은빈)
*0o*
-lxvxx-
0207/김진환
02chaeeun
0417wendy
0424S2
0min
0yoon6
0ㅂ0
0ㅇskinㅇ0
0ㅡ0a
0무다리0
0찹쌀떡0
1000soso
10대에오
123sandra
158cm
1영은
2ㅇㅎ
2땡구
301호
3대천왕
4월23일
6글자이하로
6둘리9
7v7
90hee
91laine
92SEON
940e
9탱9
@서민규
AIWD
Aiona
Alicej
Apelia
Ashby
B2ST
BORAMM
Beluga
B얄롱B
CATE
CHJJ
Chea
Crescent05
Daisyeasy
Daldal31020
Darlene
EJ1995
EZ
EchoA
Eeeeelsoup
Eighteeny
Elice
Epa
EricaL
H.E.R
HS멘톨
HahaHaa
Haileykim
Happyyy
Hch10
Heen
HeyJude
H땅콩
IOI김도연
J+
J3pop
JJEOLL
JazzTango
Jina75
Jisuyh
Joanne0803
KJHKKY
Kayhb
KimSG
K현G
LOVEjk
LadyM
Lanaa
Leeah
Lenaa
Leuk
Liva
Lucy최
Luna:D
LyuHwa
Lyyyyyyy
MAJ0319
MING__
MKella
MSG필
Miimyoo
Milli
Misoro
Nari
Nava
Oh5
OhWonder
Onlyv
PIGMARU
Paaial
Qsun
RISINGHWA
Raz
Reminne
Reumicube
Rnee
Rosalie
Ryu99
S03
SAEUS2
SML
Sjih
Sohia
StellaChoi
TRBL
TnrHus
Twices
WASA
Wendy97
YALCS
YJ0728
YJI
YeoineB
Ysseul
ZIYO
[치비문]
aaaaayo
abl.JE
adore


화장품사는년
화장품사러옴
화장품콜렉터
화장하는아기곰
화장화장해
화초속온실
화해펭귄
화히니듀
환경설정
환타
황동주
황제민현충성
회원21
회원가입ㅎㅎ
효쁭
효오니
효정0219
효정임
후라이계란
후웬
후유증
후힝
훤쨩
휘달
휘바람
휴휴후
휴휴휴
흐암
흠냐냥이
흥이
희망눈나
희뭉이
흰누니
흰발바닥
흰소
히교이
히딩
히룡
히메나무
히수히수히수7
히오니이
히지ㅎㅎ
히진이
히쿠
히토리마
히트다히트
힘내
힙업할꺼야
힝시


## ■name_list : 사용자 3096명(리뷰 2개 이상) 리스트[]

In [47]:
# 사용자 수 만큼 반복한다.
for user_key in df_dict:  # user_key : 각각의 유저이름
#     print(user_key)
    name_list.append(user_key)
    
    # 현재 사용자가 본 화장품 목록을 set에 담는다.
    for cos_key in df_dict[user_key] :   # 각각의 유저이름에 해당하는 각각의 제품이름
        # print(user_key, ":", cos_key)
        cos_set.add(cos_key)        

## ■cos_set : 제품명 99개 set()

In [48]:
print(len(cos_set))
cos_set

99


{'AHS 모이스처+옥시전 샴푸 (산소샴푸)',
 'PRO-V 극손상 케어 샴푸',
 '[단백질샴푸] 알베르 헤어 샴푸',
 '[단종] 그린 허브 레시피 샴푸 모히또',
 '[단종] 데미지케어 샴푸NC',
 '[단종] 로즈마리 스칼프 스케일링 샴푸',
 '[단종] 민트 프레쉬 샴푸',
 '[단종] 아로마 리페어 샴푸',
 '[단종] 아르간 에센셜 딥 케어 샴푸',
 '[단종] 엘레강스&센슈얼 퍼퓸 샴푸',
 '[단종] 카마 콤바',
 '[단종] 퍼퓸 샴푸 뷰티플 블루밍',
 '[단종] 포티샤 스티뮬레이팅 샴푸',
 '[단종] 퓨어스마트 샴푸',
 '[단종] 허브 그린 내추럴 헤어 샴푸',
 '가려운 두피케어 샴푸',
 '그린 허브 레시피 샴푸 (두피 클렌징 모히또 샴푸)',
 '그린티민트 프레시 샴푸',
 '내추럴 모로코 아르간 오일 샴푸',
 '내추럴 발효초 샴푸',
 '내추럴 체리블라썸 샴푸',
 '내츄럴 컬러 드라이 샴푸',
 '네이처링 리프레싱 샴푸',
 '녹차실감 샴푸액 중건성용',
 '녹차실감 샴푸액 지성용',
 '뉴',
 '대디-오',
 '데미지 클리닉 샴푸',
 '데미지케어 극손상 샴푸',
 '두피맑음 제주무환자 샴푸',
 '드라이샴푸 라이트&브리지 후레쉬',
 '드라이샴푸 클린&클래식 오리지널',
 '딥 클린 프레쉬 지성샴푸',
 '딥클렌징 샴푸',
 '로우 샴푸',
 '로즈마리 스칼프 스케일링 샴푸',
 '로즈메리 민트 퓨리파잉 샴푸',
 '리뉴잉 아르간오일 오브 모로코 샴푸',
 '리쥬베네이팅 체리블로썸 진생 샴푸',
 '리햅',
 '망고버터 뉴트리션 샴푸',
 '멘톨 쿨링 프레쉬 샴푸',
 '모링가 리페어링 샴푸',
 '바오밥 지성용 안티 오일리 샴푸',
 '벨벳 프로틴 본드 샴푸(뉴트리 인젝션 단백질 본드 샴푸/약산성)',
 '볼륨터치 샴푸',
 '블루밍 퍼퓸 화이트 릴리 샴푸',
 '비오틴B-컴플렉스 샴푸',
 '빅',
 '빠른 건조 클리닉 샴푸',
 '새티니크 글로시 리페어 샴푸',
 '샤이닝 샴푸NC',
 '스칼프 케어 샴푸',
 

## ■cos_list : 제품명 99개 리스트[]

In [49]:
cos_list = list(cos_set)

print(len(cos_list))
cos_list

99


['탈모완화 샴푸(힘없는 모발용)',
 '바오밥 지성용 안티 오일리 샴푸',
 '데미지케어 극손상 샴푸',
 '벨벳 프로틴 본드 샴푸(뉴트리 인젝션 단백질 본드 샴푸/약산성)',
 '프로페셔널 디펜스 샴푸',
 '펄 샤이닝 모이스처 샴푸',
 '딥클렌징 샴푸',
 '에스까르고 오일 샴푸',
 '내츄럴 컬러 드라이 샴푸',
 '실크리페어 리커버리 샴푸',
 '아임 트루 내추럴 샴푸',
 '빅',
 '[단종] 엘레강스&센슈얼 퍼퓸 샴푸',
 '[단종] 퍼퓸 샴푸 뷰티플 블루밍',
 '실리콘무첨가 리페어 샴푸',
 '안티에이징 샴푸',
 '로즈메리 민트 퓨리파잉 샴푸',
 '딥 클린 프레쉬 지성샴푸',
 '대디-오',
 '[단종] 데미지케어 샴푸NC',
 '알로에 허브 샴푸',
 '[단종] 카마 콤바',
 '퍼퓸 러브미 샴푸',
 '핑크 로즈 샴푸',
 '토탈리페어5 샴푸',
 '옐로우 카모마일컬렉션 샴푸',
 '내추럴 모로코 아르간 오일 샴푸',
 '인바티™ 엑스폴리에이팅 샴푸',
 '빠른 건조 클리닉 샴푸',
 '[단종] 아로마 리페어 샴푸',
 '[단종] 민트 프레쉬 샴푸',
 '내추럴 체리블라썸 샴푸',
 '쿨멘솔 샴푸',
 '가려운 두피케어 샴푸',
 '허니 앤 마카다미아 네이처샴푸 베이비파우더',
 '실리콘 프리 안티 댄드러프 샴푸',
 '리햅',
 '인텐시브 데미지케어 샴푸',
 '뉴',
 '[단종] 허브 그린 내추럴 헤어 샴푸',
 'AHS 모이스처+옥시전 샴푸 (산소샴푸)',
 '[단백질샴푸] 알베르 헤어 샴푸',
 '[단종] 퓨어스마트 샴푸',
 '프리미엄 샴푸',
 '리쥬베네이팅 체리블로썸 진생 샴푸',
 '드라이샴푸 클린&클래식 오리지널',
 '[단종] 포티샤 스티뮬레이팅 샴푸',
 '앱솔루트 샴푸',
 '샤이닝 샴푸NC',
 '콜츠푸트 안티-댄드러프 샴푸',
 '진기 샴푸',
 '아쿠아 퓨어 샴푸',
 '로우 샴푸',
 '윌로우바크 트리트먼트 샴푸',
 '[단종] 그린 허브 레시피 샴푸 모히또',
 '애플 프레쉬 샴푸',
 '폴리젠 샴푸',


In [50]:
print(len(name_list), name_list)    # 사용자 3096명 리스트

3096 [7, 7.7, 10, 92, 272, 726, 829, 1119, 1262, 2221, 2976, 10000, 11077, 20202, 28428, 43586, 43811, 101036, 221324, 357126, 373737, 9999999, 64946458, '#soso', '#달덩이', '(박은빈)', '*0o*', '-lxvxx-', '0207/김진환', '02chaeeun', '0417wendy', '0424S2', '0min', '0yoon6', '0ㅂ0', '0ㅇskinㅇ0', '0ㅡ0a', '0무다리0', '0찹쌀떡0', '1000soso', '10대에오', '123sandra', '158cm', '1영은', '2ㅇㅎ', '2땡구', '301호', '3대천왕', '4월23일', '6글자이하로', '6둘리9', '7v7', '90hee', '91laine', '92SEON', '940e', '9탱9', '@서민규', 'AIWD', 'Aiona', 'Alicej', 'Apelia', 'Ashby', 'B2ST', 'BORAMM', 'Beluga', 'B얄롱B', 'CATE', 'CHJJ', 'Chea', 'Crescent05', 'Daisyeasy', 'Daldal31020', 'Darlene', 'EJ1995', 'EZ', 'EchoA', 'Eeeeelsoup', 'Eighteeny', 'Elice', 'Epa', 'EricaL', 'H.E.R', 'HS멘톨', 'HahaHaa', 'Haileykim', 'Happyyy', 'Hch10', 'Heen', 'HeyJude', 'H땅콩', 'IOI김도연', 'J+', 'J3pop', 'JJEOLL', 'JazzTango', 'Jina75', 'Jisuyh', 'Joanne0803', 'KJHKKY', 'Kayhb', 'KimSG', 'K현G', 'LOVEjk', 'LadyM', 'Lanaa', 'Leeah', 'Lenaa', 'Leuk', 'Liva', 'Lucy최', 'Luna:D', '

In [51]:
print(len(cos_list), cos_list)    # 제품명 99개 리스트

99 ['탈모완화 샴푸(힘없는 모발용)', '바오밥 지성용 안티 오일리 샴푸', '데미지케어 극손상 샴푸', '벨벳 프로틴 본드 샴푸(뉴트리 인젝션 단백질 본드 샴푸/약산성)', '프로페셔널 디펜스 샴푸', '펄 샤이닝 모이스처 샴푸', '딥클렌징 샴푸', '에스까르고 오일 샴푸', '내츄럴 컬러 드라이 샴푸', '실크리페어 리커버리 샴푸', '아임 트루 내추럴 샴푸', '빅', '[단종] 엘레강스&센슈얼 퍼퓸 샴푸', '[단종] 퍼퓸 샴푸 뷰티플 블루밍', '실리콘무첨가 리페어 샴푸', '안티에이징 샴푸', '로즈메리 민트 퓨리파잉 샴푸', '딥 클린 프레쉬 지성샴푸', '대디-오', '[단종] 데미지케어 샴푸NC', '알로에 허브 샴푸', '[단종] 카마 콤바', '퍼퓸 러브미 샴푸', '핑크 로즈 샴푸', '토탈리페어5 샴푸', '옐로우 카모마일컬렉션 샴푸', '내추럴 모로코 아르간 오일 샴푸', '인바티™ 엑스폴리에이팅 샴푸', '빠른 건조 클리닉 샴푸', '[단종] 아로마 리페어 샴푸', '[단종] 민트 프레쉬 샴푸', '내추럴 체리블라썸 샴푸', '쿨멘솔 샴푸', '가려운 두피케어 샴푸', '허니 앤 마카다미아 네이처샴푸 베이비파우더', '실리콘 프리 안티 댄드러프 샴푸', '리햅', '인텐시브 데미지케어 샴푸', '뉴', '[단종] 허브 그린 내추럴 헤어 샴푸', 'AHS 모이스처+옥시전 샴푸 (산소샴푸)', '[단백질샴푸] 알베르 헤어 샴푸', '[단종] 퓨어스마트 샴푸', '프리미엄 샴푸', '리쥬베네이팅 체리블로썸 진생 샴푸', '드라이샴푸 클린&클래식 오리지널', '[단종] 포티샤 스티뮬레이팅 샴푸', '앱솔루트 샴푸', '샤이닝 샴푸NC', '콜츠푸트 안티-댄드러프 샴푸', '진기 샴푸', '아쿠아 퓨어 샴푸', '로우 샴푸', '윌로우바크 트리트먼트 샴푸', '[단종] 그린 허브 레시피 샴푸 모히또', '애플 프레쉬 샴푸', '폴리젠 샴푸', '자윤비책 백단향 샴푸', '데미지 클리닉 샴푸', '실크리페어 샤이닝 샴푸', '망고버터 뉴트

# 2. 학습을 위한 index 딕셔너리 만들기

In [52]:
# 학습할 데이터를 준비한다.
rating_dic = {
    'user' : [],
    'product' : [],
    'rating' : []
    }

## ■rating_dict 만들기(인덱스 모음 딕셔너리)
### {'user': [유저리스트], 'product': [제품리스트], 'rating': [평점리스트]}

In [53]:
rating_dic

{'user': [], 'product': [], 'rating': []}

In [54]:
for user_key in df_dict :  # user_key : 각각의 유저이름
    print(user_key)

7
7.7
10
92
272
726
829
1119
1262
2221
2976
10000
11077
20202
28428
43586
43811
101036
221324
357126
373737
9999999
64946458
#soso
#달덩이
(박은빈)
*0o*
-lxvxx-
0207/김진환
02chaeeun
0417wendy
0424S2
0min
0yoon6
0ㅂ0
0ㅇskinㅇ0
0ㅡ0a
0무다리0
0찹쌀떡0
1000soso
10대에오
123sandra
158cm
1영은
2ㅇㅎ
2땡구
301호
3대천왕
4월23일
6글자이하로
6둘리9
7v7
90hee
91laine
92SEON
940e
9탱9
@서민규
AIWD
Aiona
Alicej
Apelia
Ashby
B2ST
BORAMM
Beluga
B얄롱B
CATE
CHJJ
Chea
Crescent05
Daisyeasy
Daldal31020
Darlene
EJ1995
EZ
EchoA
Eeeeelsoup
Eighteeny
Elice
Epa
EricaL
H.E.R
HS멘톨
HahaHaa
Haileykim
Happyyy
Hch10
Heen
HeyJude
H땅콩
IOI김도연
J+
J3pop
JJEOLL
JazzTango
Jina75
Jisuyh
Joanne0803
KJHKKY
Kayhb
KimSG
K현G
LOVEjk
LadyM
Lanaa
Leeah
Lenaa
Leuk
Liva
Lucy최
Luna:D
LyuHwa
Lyyyyyyy
MAJ0319
MING__
MKella
MSG필
Miimyoo
Milli
Misoro
Nari
Nava
Oh5
OhWonder
Onlyv
PIGMARU
Paaial
Qsun
RISINGHWA
Raz
Reminne
Reumicube
Rnee
Rosalie
Ryu99
S03
SAEUS2
SML
Sjih
Sohia
StellaChoi
TRBL
TnrHus
Twices
WASA
Wendy97
YALCS
YJ0728
YJI
YeoineB
Ysseul
ZIYO
[치비문]
aaaaayo
abl.JE
adore


화장품사는년
화장품사러옴
화장품콜렉터
화장하는아기곰
화장화장해
화초속온실
화해펭귄
화히니듀
환경설정
환타
황동주
황제민현충성
회원21
회원가입ㅎㅎ
효쁭
효오니
효정0219
효정임
후라이계란
후웬
후유증
후힝
훤쨩
휘달
휘바람
휴휴후
휴휴휴
흐암
흠냐냥이
흥이
희망눈나
희뭉이
흰누니
흰발바닥
흰소
히교이
히딩
히룡
히메나무
히수히수히수7
히오니이
히지ㅎㅎ
히진이
히쿠
히토리마
히트다히트
힘내
힙업할꺼야
힝시


## ■ rating_dic에 index(user, product, rating)로 채우기

In [55]:
# user 수만큼 반복
user_list = []
for user_key in df_dict :  # user_key : 각각의 유저이름
    user_list.append(user_key)
    # 해당 사용자가 본 화장품 수 만큼 반복
    for cos_key in df_dict[user_key] :

        # 사용자 인덱스 번호를 추출한다.
        a1 = name_list.index(user_key)

        # 화장품 인덱스 번호를 추출한다.
        a2 = cos_list.index(cos_key)

        # 평점을 가져온다.
        a3 = df_dict[user_key][cos_key]

        # 딕셔너리에 담는다.
        rating_dic['user'].append(a1)
        rating_dic['product'].append(a2)
        rating_dic['rating'].append(a3)

In [56]:
print(len(user_list))
user_list

3096


[7,
 7.7,
 10,
 92,
 272,
 726,
 829,
 1119,
 1262,
 2221,
 2976,
 10000,
 11077,
 20202,
 28428,
 43586,
 43811,
 101036,
 221324,
 357126,
 373737,
 9999999,
 64946458,
 '#soso',
 '#달덩이',
 '(박은빈)',
 '*0o*',
 '-lxvxx-',
 '0207/김진환',
 '02chaeeun',
 '0417wendy',
 '0424S2',
 '0min',
 '0yoon6',
 '0ㅂ0',
 '0ㅇskinㅇ0',
 '0ㅡ0a',
 '0무다리0',
 '0찹쌀떡0',
 '1000soso',
 '10대에오',
 '123sandra',
 '158cm',
 '1영은',
 '2ㅇㅎ',
 '2땡구',
 '301호',
 '3대천왕',
 '4월23일',
 '6글자이하로',
 '6둘리9',
 '7v7',
 '90hee',
 '91laine',
 '92SEON',
 '940e',
 '9탱9',
 '@서민규',
 'AIWD',
 'Aiona',
 'Alicej',
 'Apelia',
 'Ashby',
 'B2ST',
 'BORAMM',
 'Beluga',
 'B얄롱B',
 'CATE',
 'CHJJ',
 'Chea',
 'Crescent05',
 'Daisyeasy',
 'Daldal31020',
 'Darlene',
 'EJ1995',
 'EZ',
 'EchoA',
 'Eeeeelsoup',
 'Eighteeny',
 'Elice',
 'Epa',
 'EricaL',
 'H.E.R',
 'HS멘톨',
 'HahaHaa',
 'Haileykim',
 'Happyyy',
 'Hch10',
 'Heen',
 'HeyJude',
 'H땅콩',
 'IOI김도연',
 'J+',
 'J3pop',
 'JJEOLL',
 'JazzTango',
 'Jina75',
 'Jisuyh',
 'Joanne0803',
 'KJHKKY',
 'Kayhb',
 'K

In [57]:
rating_dic

{'user': [0,
  0,
  0,
  0,
  0,
  1,
  1,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  4,
  5,
  5,
  6,
  6,
  6,
  7,
  7,
  8,
  8,
  9,
  9,
  10,
  10,
  10,
  10,
  10,
  11,
  11,
  12,
  12,
  13,
  13,
  14,
  14,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  16,
  16,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  18,
  18,
  19,
  19,
  20,
  20,
  20,
  21,
  21,
  22,
  22,
  23,
  23,
  24,
  24,
  24,
  25,
  25,
  26,
  27,
  27,
  28,
  28,
  28,
  28,
  28,
  29,
  29,
  30,
  30,
  30,
  31,
  31,
  31,
  32,
  32,
  32,
  33,
  33,
  34,
  34,
  34,
  34,
  35,
  35,
  36,
  36,
  36,
  36,
  37,
  37,
  38,
  38,
  38,
  39,
  39,
  40,
  40,
  41,
  41,
  42,
  42,
  43,
  43,
  44,
  44,
  44,
  44,
  44,
  45,
  45,
  46,
  46,
  47,
  47,
  47,
  47,
  48,
  48,
  49,
  49,
  49,
  49,
  50,
  50,
  50,
  51,
  51,
  52,
  52,
  53,
  53,
  54,
  54,
  54,
  54,
  55,
  55,
  55,
  56,
  56,
  57,
  57,
  57,
  57,
  57,
  57,
  57,
  

In [58]:
# 사용자 - 화장품 - 평점
print(len(rating_dic['user']))
print(len(rating_dic['product']))
print(len(rating_dic['rating']))

8932
8932
8932


In [59]:
# 사용자
print(len(rating_dic['user']))
print(rating_dic['user'])

8932
[0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 6, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 19, 19, 20, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 24, 25, 25, 26, 27, 27, 28, 28, 28, 28, 28, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 34, 34, 34, 34, 35, 35, 36, 36, 36, 36, 37, 37, 38, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 44, 44, 44, 45, 45, 46, 46, 47, 47, 47, 47, 48, 48, 49, 49, 49, 49, 50, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 54, 54, 55, 55, 55, 56, 56, 57, 57, 57, 57, 57, 57, 57, 57, 58, 58, 59, 59, 59, 59, 59, 59, 60, 60, 60, 61, 61, 62, 62, 62, 62, 63, 63, 64, 64, 65, 65, 65, 65, 65, 65, 66, 66, 66, 66, 66, 67, 67, 67, 67, 68, 68, 69, 69, 70, 70, 71, 71, 72, 72, 72, 72, 73, 73, 73, 73, 74, 74, 75, 75, 75, 75, 75, 75, 75, 75, 76, 76, 76, 77, 77, 77, 77, 77, 78, 78, 79, 79, 80, 80, 80, 81, 81, 81, 82, 82, 82, 82, 82, 

In [60]:
print(len(rating_dic['product']))
print(rating_dic['product'])

8932
[95, 19, 12, 46, 58, 93, 64, 64, 15, 74, 90, 16, 3, 14, 51, 25, 23, 40, 10, 41, 45, 30, 32, 56, 81, 71, 85, 9, 30, 23, 85, 14, 10, 56, 67, 62, 96, 45, 47, 35, 97, 12, 58, 88, 28, 89, 59, 15, 63, 97, 5, 67, 93, 3, 95, 14, 9, 71, 80, 50, 74, 96, 70, 2, 92, 75, 12, 6, 23, 45, 20, 69, 31, 15, 32, 92, 73, 71, 13, 96, 75, 36, 11, 69, 18, 82, 52, 9, 22, 97, 45, 91, 89, 54, 31, 57, 28, 57, 23, 63, 22, 93, 9, 15, 96, 31, 86, 2, 87, 63, 97, 74, 96, 12, 75, 86, 13, 50, 32, 22, 30, 14, 93, 71, 30, 26, 90, 70, 82, 45, 53, 1, 77, 82, 65, 64, 14, 71, 32, 54, 63, 12, 90, 84, 5, 12, 52, 67, 33, 22, 8, 56, 66, 23, 95, 19, 87, 83, 51, 98, 66, 82, 14, 45, 88, 51, 81, 7, 83, 4, 23, 54, 56, 11, 15, 25, 50, 32, 98, 58, 59, 63, 55, 67, 46, 82, 84, 57, 12, 37, 93, 23, 17, 51, 25, 86, 97, 23, 12, 13, 26, 91, 86, 41, 3, 56, 23, 12, 32, 70, 97, 62, 71, 85, 74, 29, 69, 92, 67, 29, 15, 24, 23, 75, 4, 54, 69, 31, 77, 15, 55, 74, 5, 8, 60, 79, 40, 19, 70, 83, 97, 33, 78, 18, 92, 13, 82, 87, 54, 42, 77, 62, 18, 1

In [61]:
print(len(rating_dic['rating']))
print(rating_dic['rating'])

8932
[2, 3, 2, 4, 4, 5, 5, 4, 3, 2, 4, 4, 4, 5, 4, 3, 3, 5, 3, 2, 4, 4, 4, 5, 4, 3, 3, 2, 4, 3, 3, 2, 2, 3, 3, 4, 2, 3, 4, 4, 1, 4, 4, 5, 3, 3, 4, 3, 3, 2, 4, 5, 5, 4, 3, 2, 4, 3, 4, 3, 3, 4, 3, 5, 4, 4, 1, 5, 4, 4, 5, 3, 1, 4, 4, 1, 3, 1, 5, 5, 5, 5, 4, 5, 5, 5, 2, 3, 2, 4, 4, 4, 5, 5, 2, 2, 4, 4, 2, 5, 3, 4, 2, 3, 3, 3, 5, 3, 3, 2, 3, 2, 4, 2, 5, 2, 4, 1, 4, 5, 5, 4, 2, 2, 4, 2, 2, 3, 1, 2, 2, 5, 3, 5, 3, 3, 1, 4, 2, 3, 1, 1, 5, 4, 1, 4, 2, 4, 5, 1, 3, 4, 4, 4, 2, 4, 3, 4, 3, 4, 4, 4, 4, 4, 3, 4, 5, 5, 3, 5, 3, 4, 3, 2, 2, 5, 4, 2, 2, 4, 4, 4, 4, 5, 4, 5, 5, 5, 3, 3, 5, 5, 2, 5, 3, 1, 2, 3, 5, 2, 3, 4, 4, 4, 4, 3, 4, 3, 4, 4, 3, 5, 3, 2, 2, 3, 2, 4, 5, 4, 2, 1, 3, 3, 4, 4, 3, 3, 5, 5, 4, 4, 4, 4, 3, 3, 5, 3, 4, 4, 5, 3, 2, 4, 4, 2, 3, 3, 4, 5, 5, 4, 5, 4, 3, 4, 4, 2, 4, 1, 1, 5, 5, 4, 4, 1, 3, 4, 4, 4, 4, 4, 5, 3, 3, 4, 4, 5, 4, 1, 4, 3, 2, 3, 2, 3, 1, 5, 2, 5, 3, 4, 4, 3, 3, 3, 2, 2, 4, 3, 3, 1, 3, 1, 3, 1, 1, 5, 4, 5, 4, 5, 2, 2, 5, 5, 4, 3, 4, 1, 5, 5, 4, 5, 4, 3, 4, 5, 4, 3, 4, 1

In [62]:
#  데이터셋을 만든다

In [63]:
df_index = pd.DataFrame(rating_dic)

In [64]:
df_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 3 columns):
user       8932 non-null int64
product    8932 non-null int64
rating     8932 non-null int64
dtypes: int64(3)
memory usage: 209.4 KB


In [65]:
for i in tqdm_notebook(range(len(df_index))):
    df_index['user'][i] = str(df_index['user'][i])

In [66]:
print(df_index.shape)
df_index

(8932, 3)


,user,product,rating
0,0,95,2
1,0,19,3
2,0,12,2
3,0,46,4
4,0,58,4
5,1,93,5
6,1,64,5
7,2,64,4
8,2,15,3
9,2,74,2


In [67]:
df_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8932 entries, 0 to 8931
Data columns (total 3 columns):
user       8932 non-null int64
product    8932 non-null int64
rating     8932 non-null int64
dtypes: int64(3)
memory usage: 209.4 KB


## -> index 딕셔너리 생성 완료!

# 3. 학습 시키기

### surprise 모델

In [68]:
import surprise
# 데이터를 읽어들이는 객체 생성 (rating_scale : 평점의 범위)
reader = surprise.Reader(rating_scale = (1, 5))
reader

In [69]:
col_list = ['user', 'product', 'rating']

data = surprise.Dataset.load_from_df(df_index[col_list], reader)

In [70]:
print(col_list)
print(data)

['user', 'product', 'rating']


In [71]:
from surprise import SVD, evaluate
from surprise import NMF
from surprise import KNNBasic

In [72]:
# 학습한다.
trainset = data.build_full_trainset()
option = {'name' : 'pearson'}
algo = surprise.KNNBasic(sim_options=option)

algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [73]:
from surprise.model_selection import cross_validate

## CV(교차검증)로 accuracy 측정
## http://surpriselib.com/ 참고

In [76]:
cross_validate(algo, data)["test_mae"].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.9612759761051006

In [75]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.1624  1.1477  1.1553  1.1848  1.139

{'test_rmse': array([1.16239139, 1.14771217, 1.15525594, 1.18475273, 1.1390791 ,
        1.17305527, 1.18175528, 1.16979135, 1.13282732, 1.17738034]),
 'test_mae': array([0.96859794, 0.95728495, 0.97096239, 0.97751772, 0.95234032,
        0.98338976, 0.98413037, 0.98463601, 0.94826305, 0.97654257]),
 'fit_time': (1.1432840824127197,
  1.117063045501709,
  1.1192240715026855,
  1.2822318077087402,
  1.14013671875,
  1.2360939979553223,
  1.1530811786651611,
  1.1093776226043701,
  1.1235127449035645,
  1.1146047115325928),
 'test_time': (0.11730480194091797,
  0.12466597557067871,
  0.12180471420288086,
  0.1653304100036621,
  0.10969376564025879,
  0.12664580345153809,
  0.13061833381652832,
  0.12833476066589355,
  0.11469268798828125,
  0.11763620376586914)}

In [80]:
for i in tqdm_notebook(range(len(name_list))):
    name_list[i] = str(name_list[i])

In [84]:
# "사용자1"의 화장품을 추천받는다.
who = input('닉네임을 입력해주세요: ')
print('\n')

index = name_list.index(who)
print('user_index: ',index)
print('\n')

result = algo.get_neighbors(index, k=5)
print('당신의 피부와 유사한 사용자는? : ', result)
print('\n')

# user 귯니스에 대해 화장품을 추천한다.
print('당신에게 추천드리는 화장품 : ', '\n')

for r1 in result :
    max_rating=data.df[data.df["user"]==r1]["rating"].max()
    cos_id=data.df[(data.df["rating"]==max_rating)&(data.df["user"]==r1)]["product"].values
    
    for cos_item in cos_id:
        print(cos_list[cos_item])

닉네임을 입력해주세요: 7


user_index:  0


당신의 피부와 유사한 사용자는? :  [65, 820, 1810, 2993, 1]


당신에게 추천드리는 화장품 :  

아쿠아 퓨어 샴푸
스칼프 케어 샴푸
실리콘무첨가 리페어 샴푸
티트리 퓨리파잉 샴푸
벨벳 프로틴 본드 샴푸(뉴트리 인젝션 단백질 본드 샴푸/약산성)
프리즈 이즈 미라큘러스 리커버리 샴푸
[단종] 아르간 에센셜 딥 케어 샴푸
녹차실감 샴푸액 중건성용


In [90]:
df

,user,product,rating_one
3,내제잘,실크리페어 리커버리 샴푸,2
5,수정222,내추럴 발효초 샴푸,2
6,미묘뮤몌,퍼퓸 퓨어 브리즈 샴푸,1
7,속이알차니,퍼펙트 리페어 샴푸,1
8,볼뤼,그린티민트 프레시 샴푸,5
9,방노방노,퍼펙트 세럼 샴푸 오리지널,2
10,랑꿍,실크리페어 샤이닝 샴푸,3
12,tjdudd,허니 앤 마카다미아 네이처샴푸 베이비파우더,5
13,이브엄마,드라이샴푸 클린&클래식 오리지널,2
15,꽃집여자,녹차실감 샴푸액 지성용,3


In [92]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(df[['user', 'product', 'rating_one']], reader)

In [93]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [102]:
from surprise import SVD, accuracy
option = {'name' : 'pearson'}
algo = surprise.KNNBasic(sim_options=option)

algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [103]:
predictions = algo.test(testset)

In [104]:
from surprise import accuracy
accuracy.rmse(predictions)

RMSE: 1.1374


1.1374468415391275